# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [117]:
class Matrix:
    def __init__(self, n = None, m = None, values = None):
        if values is not None:
            if not all(len(row) == len(values[0]) for row in values):
                raise ValueError("The rows are not the same length")
            self.data = [row[:] for row in values]
            self.rows = len(values)
            self.columns = len(values[0])
        elif n is not None and m is not None:
            self.rows = n
            self.columns = m
            self.data = [[0 for i in range(m)] for i in range(n)]
        else:
            raise ValueError("Error: invalid input")

    def __getitem__(self, key):
        if isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        else:
            self.data[key] = value

    def __eq__(self, matrix2):
        return self.data == matrix2.data

    def __repr__(self):
        return "\n".join([" ".join(map(str,row)) for row in self.data])

    def shape(self):
        return self.rows, self.columns

    def transpose(self):
        return Matrix(self.columns, self.rows, [[self.data[i][j] for i in range(self.rows)] for j in range(self.columns)])

    def row(self, n):
        return Matrix(1, self.columns, [self.data[n]])

    def column(self, n):
        column = [self.data[i][n] for i in range(self.rows)]
        return Matrix(self.rows, 1, [[item] for item in column])

    def to_list(self):
        return self.data

    def block(self, n0, n1, m0, m1):
        blockdata = [row[m0:m1+1] for row in self.data[n0:n1+1]]
        return Matrix(n1 - n0 + 1, m1 - m0 + 1, blockdata)

    def scalarmult(self,c):
        return Matrix(self.rows, self.columns, [[self.data[i][j] * c for j in range(self.columns)] for i in range(self.rows)])

    def add(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices are not the same shape")
        return Matrix(self.rows, self.columns, [[self.data[i][j] + N[i][j] for j in range(self.columns)] for i in range(self.rows)])

    def sub(self,N):
        if self.shape() != N.shape():
            raise ValueError("Matrices are not the same shape")
        return Matrix(self.rows, self.columns, [[self.data[i][j] - N[i][j] for j in range(self.columns)] for i in range(self.rows)])

    def matmult(self, N):
        if self.columns != N.rows:
            raise ValueError("Matricies are not of the correct shape to multiply")
        product = []
        for i in range(self.rows):
            row = []
            for j in range(N.columns):
                prod = sum(self.data[i][k] * N[k][j] for k in range(self.columns))
                row.append(prod)
            product.append(row)
        return Matrix(self.rows, N.columns, product)

    def elementmult(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices are not the same shape")
        return Matrix(self.rows, self.columns, [[self.data[i][j] * N[i][j] for j in range(self.columns)] for i in range(self.rows)])

    def equals(self, N):
        if self.shape() != N.shape():
            return False
        return all(self.data[i][j] == N[i][j] for i in range (self.rows) for j in range(self.columns))

    def __add__(self, other):
        if isinstance(other, (int,float)):
            return Matrix(self.rows, self.columns, [[self[i][j] + other for j in range(self.columns)] for i in range(self.rows)])
        else:
            return self.add(other)

    def __sub__(self, N):
        return self.sub(N)

    def __mul__(self,other):
        if isinstance(other, Matrix):
            return self.matmult(other)
        else:
            return self.scalarmult(other)

    def __rmul__(self, c):
        return self.scalarmult(c)

    def __eq__(self, N):
        return self.equals(N)

def constant(n, m, c):
    return Matrix(n, m, [[c for i in range(m)] for i in range(n)])

def zeros(n,m):
    return Matrix(n, m, [[0.0 for i in range(m)] for i in range(n)])

def ones(n,m):
    return Matrix(n, m, [[1.0 for i in range(m)] for i in range(n)])

def eye(n):
    return Matrix(n, n, [[1 if i == j else 0 for j in range(n)] for i in range(n)])

In [ ]:
## Tests of the code start below this line

In [47]:
# Creating a matrix, calling the value of a cell, then changing that cell's value
M1 = Matrix(values=[[1,2,3],[4,5,6],[7,8,9]])
print(M1[1][2])
M1[1][2] = 10
print(M1[1][2])

6
10


In [48]:
# Making a blank matrix equal to another matrix
M2 = Matrix(3,3)
print(M2[0,2])
M2 = M1
print(M2[0,2])

0
3


In [49]:
# Using __repr__ to print M2
print(M2)

1 2 3
4 5 10
7 8 9


In [70]:
# Using shape(), transpose(), row(n), column(n), to_list(), and block(n0,n1,m0,m1)

M3 = Matrix(values=[[1,2,3,4],[3,4,5,6],[5,6,7,8],[7,8,9,10]])
print(M3, "\n")
print(M3.shape(), "\n")
print(M3.transpose(), "\n")
print(M3.row(2), "\n")
print(M3.column(1), "\n")
print(M3.to_list(), "\n")
print(M3.block(0,1,0,1))

1 2 3 4
3 4 5 6
5 6 7 8
7 8 9 10 

(4, 4) 

1 3 5 7
2 4 6 8
3 5 7 9
4 6 8 10 

5 6 7 8 

2
4
6
8 

[[1, 2, 3, 4], [3, 4, 5, 6], [5, 6, 7, 8], [7, 8, 9, 10]] 

1 2
3 4


In [87]:
# Testing the constant, zero, one, and identity matrices
Mc = constant(3,4,8)
print(Mc, "\n")

Mzeros = zeros(5,2)
print(Mzeros, "\n")

Mones = ones(3,4)
print(Mones, "\n")

Midentity = eye(4)
print(Midentity, "\n")

8 8 8 8
8 8 8 8
8 8 8 8 

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0 

1.0 1.0 1.0 1.0
1.0 1.0 1.0 1.0
1.0 1.0 1.0 1.0 

1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1 



In [116]:
# Testing scalarmult, add, sub, matmult, elementmult, and equals
M4 = Matrix(values = [[1,2,3],[4,5,6],[7,8,9]])
M5 = Matrix(values = [[3,6,3],[5,9,8],[1,4,2]])

print(M4, "\n")
print(M5, "\n")
print(M4.scalarmult(3), "\n")
print(M4.add(M5), "\n")
print(M4.sub(M5), "\n")
print(M4.matmult(M5), "\n")
print(M4.elementmult(M5), "\n")
print(M4.equals(M5), "\n")

1 2 3
4 5 6
7 8 9 

3 6 3
5 9 8
1 4 2 

3 6 9
12 15 18
21 24 27 

4 8 6
9 14 14
8 12 11 

-2 -4 0
-1 -4 -2
6 4 7 

16 36 25
43 93 64
70 150 103 

3 12 9
20 45 48
7 32 18 

False 



In [119]:
# Testing the overloaded functions

M6 = Matrix(values = [[1,2,3],[4,5,6],[7,8,9]])
M7 = Matrix(values = [[6,2,3],[5,4,1],[3,6,7]])
M8 = Matrix(3,3)

print(M6, "\n")
print(M7, "\n")
print(2 * M6, "\n")
print(M6 * 2, "\n")
print(M6 + M7, "\n")
print(M6 - M7, "\n")
print(M6 * M7, "\n")
print(M6 == M7, "\n")
M8 = M6
print(M8, "\n")


1 2 3
4 5 6
7 8 9 

6 2 3
5 4 1
3 6 7 

2 4 6
8 10 12
14 16 18 

2 4 6
8 10 12
14 16 18 

7 4 6
9 9 7
10 14 16 

-5 0 0
-1 1 5
4 2 2 

25 28 26
67 64 59
109 100 92 

False 

1 2 3
4 5 6
7 8 9 



In [124]:
# (AB)C = A(BC)

A = Matrix(values = [[1,2],[3,4]])
B = Matrix(values = [[7,3],[4,8]])
C = Matrix(values = [[10,10],[10,10]])

AB = A * B
ABC1 = AB * C
BC = B * C
ABC2 = A * BC
print(ABC1, "\n")
print(ABC2, "\n")

340 340
780 780 

340 340
780 780 



In [126]:
# A(B + C) = AB + AC

BpC = B + C
ABpC = A * BpC

AB = A * B
AC = A * C
ABpAC = AB + AC

print(ABpC, "\n")
print(ABpAC)


45 49
107 111 

45 49
107 111


In [127]:
# AB != BA

AB = A * B
BA = B * A

print(AB,"\n")
print(BA)

15 19
37 41 

16 26
28 40


In [128]:
# AI = A

I = Matrix(values = [[1,0],[0,1]])

AI = A * I

print(AI, "\n")
print(A)

1 2
3 4 

1 2
3 4
